In [ ]:
#@title Googleドライブにマウントする
from google.colab import drive
drive.mount('/content/drive')

Googleドライブのマイドライブ内に解析専用のフォルダを作成する。

ここではEMGとする。そして下のworking_directoryを合わせる。

In [ ]:
mydrive = "/content/drive/MyDrive/"
#@title フォルダの設定
working_directory = "EMG" #@param {type:"string"}

In [26]:
#@title EMGを計測後センサから取り出したデータファイルの入ったフォルダ
folder_name = "mem-20211115-131617" #@param {type:"string"}
folderwithfile = mydrive+working_directory+"/"+folder_name+"/"

In [15]:
#@title 解析を始めるポイントの決定(5秒以降がよい)
start = 5100 #@param {type:"slider", min:0, max:10000, step:100}

In [18]:
#@title ココはこのまま実行
import os
import csv
import pandas as pd
import datetime
import glob
import numpy as np

os.chdir(mydrive+working_directory)
import emg
print(f"現在の作業ディレクトリ:{os.getcwd()}")

現在の作業ディレクトリ:/content/drive/MyDrive/EMG


In [25]:
#@title データの読み込みと処理後のデータの保存 { display-mode: "form" }
filename = glob.glob(folderwithfile+"mem-*.csv")
dat = pd.read_csv(filename[0],header=None)

bandpasslow = 15 
bandpasshigh = 350 

# EMG1
myd0 = dat.iloc[start:,2].to_numpy()
myd0_band = emg.bpf(myd0,1000,bandpasslow,bandpasshigh)
myd0_abs = np.abs(myd0_band)
val = emg.integral(myd0_abs,0,start,len(myd0_abs)-2)
for name,dd in zip(["rowdata","band","abs","integ"],[myd0,myd0_band,myd0_abs,val]):
    with open(f'{name}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(dd.tolist())